## Gemma Text Generation

url: https://huggingface.co/google/gemma-2-2b-it

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import uuid
import pandas as pd
import kagglehub
from transformers import AutoTokenizer, AutoModelForCausalLM

To use Gemma:


*   Accept Google's Terms & Conditions when prompted
*   Create [HuggingFace token](https://huggingface.co/settings/tokens) with name HF_TOKEN
  * Allow Read access to contents of all public gated repos you can access
* In Colab: Secrets (key symbol on left) --> Name: HF_TOKEN; Value: your token value
* ALTERNATIVELY: add `hf auth login` line of code (haven't tried this but it should work & is easier than creating a token)



In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [3]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNo

In [4]:
# Test inference
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "who is michael jordan?"}
        ]
    },
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Michael Jordan is widely considered one of the greatest basketball players of all time. 

Here's a summary:

* **Full Name:** Michael Jeffrey Jordan
* **Born:** February 1


In [5]:
# Create test dataset
gemma_df = pd.DataFrame(columns=['uuid', 'generated_article'])

In [26]:
def generate_article():
  messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": f"Write the body of a news article, do not include a headline or title. It can be about any topic"}
        ]
    },
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
  ).to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=1.1, top_p=0.95,top_k=50)
  response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
  return response

In [27]:
# Generate articles
n = 10 # Number of datapoints to create
for i in range(0, n + 1):
  # Add unique identifier for the row
  gemma_df.loc[i, 'uuid'] = str(uuid.uuid4())
  response = generate_article()
  gemma_df.loc[i, 'generated_article'] = response

In [28]:
gemma_df.head()

,uuid,generated_article
0,08f3d500-d967-4f95-9c6e-6ee8ef9a549d,The community of Oakhaven woke up to a surreal...
1,5da41b40-4793-4cfd-9458-1c84cb528005,The annual River City Balloon Glow was a resou...
2,5f95563d-5c07-401f-84ad-4c712494151d,"The recent surge in popularity of ""cottagecore..."
3,7fc91c3b-2b21-458d-86de-c94caacd8eb3,The city of Seattle's annual Emerald City Coll...
4,1bbac289-e94f-45bc-b880-ff9d9e1735aa,The annual Global Robotics Forum kicked off in...


In [39]:
for i in range(10):
  print(i, gemma_df['generated_article'][i])

0 The community of Oakhaven woke up to a surreal sight this morning: the ancient Willow Creek, long considered a tranquil centerpiece of the town, was filled with an impossibly pink and swirling mass of bubbles. While many saw it as nothing more than an unusual atmospheric phenomenon, a handful of residents were deeply disturbed.  Local artist, Eleanor Wright, claims it to be a message: "it's a sign," she said, standing in her garden surrounded by shimmering bubbles. "The water is crying
1 The annual River City Balloon Glow was a resounding success this weekend, drawing crowds of over 5,000 people to River City Park.  Colorful hot air balloons illuminated the evening sky as attendees enjoyed live music, food vendors, and family-friendly activities.  Many families came out with their little ones, holding onto their balloons with a mix of excitement and wonder as they bounced and soared through the air.  Mayor Thompson, who served as honorary balloon pilot, remarked, "It's
2 The recent s

In [ ]:
gemma_df.to_csv("gemma_outputs.csv")